In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.cluster import hierarchy

In [8]:
dfmovies = pd.read_csv('../Datasets/Movies/movies_description.txt', sep='|', header=None)
dfusers = pd.read_csv('../Datasets/Movies/users_description.txt', sep='|', header=None)
dfscores = pd.read_csv('../Datasets/Movies/movies_users.txt', sep='\t', header=None)

In [9]:
dfmovies.columns = ['id', 'title', 'date', 'NA', 'link', 'Unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
genres = ['Unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
moviesT = dfmovies.values[:,1]

In [10]:
nusers = len(dfusers.values)
nmovies = len(dfmovies.values)
evaluations = np.zeros((nmovies,nusers))
evaluations[dfscores.values[:,1] - 1, dfscores.values[:,0] - 1] = dfscores.values[:,2]
avgs = []
for i in range(len(evaluations)):
    avgs.append((np.sum(evaluations[i]) / np.sum(evaluations[i,:] !=0)) / 5)

df = dfmovies[genres].copy()
df['Avg'] = avgs
df.index = moviesT
df

,Unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Avg
Toy Story (1995),0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.775664
GoldenEye (1995),0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.641221
Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.606667
Get Shorty (1995),0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0.710048
Copycat (1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.660465
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mat' i syn (1997),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.200000
B. Monkey (1998),0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0.600000
Sliding Doors (1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0.400000
You So Crazy (1994),0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.600000


In [11]:
Z = hierarchy.linkage(df, method = 'ward')

In [6]:
"""
plt.figure(figsize=(15,15))
hierarchy.set_link_color_palette(['plum','lightseagreen', 'silver'])
dn = hierarchy.dendrogram(Z, labels=moviesT)
plt.show()
"""

"\nplt.figure(figsize=(15,15))\nhierarchy.set_link_color_palette(['plum','lightseagreen', 'silver'])\ndn = hierarchy.dendrogram(Z, labels=moviesT)\nplt.show()\n"

### Print the list of movies id's and titles

In [ ]:
for i in range(len(dfmovies)):
    print(dfmovies['id'][i], dfmovies['title'][i])

### Ask the user to write the ids of five movies that he/she likes

In [12]:
favMovies = list(map(int,input("Write 5 ID's of your favourite movies: ").strip().split()))[:5]

Write 5 ID's of your favourite movies: 1 29 35 50 55


### Recommend the user other five movies (different from the user list)

In [13]:
#favMovies = [1, 29, 35, 50, 55, 71, 82, 91, 95, 288, 543, 993]
print("Your favorite movies are:")
[print("", moviesT[i - 1], sep = '\t') for i in favMovies]
print('\nWe think you might like:')
alreadyRecommended = []
for m in favMovies:
    foundMovie = False
    threshold = 0.0
    while(True):
        g = hierarchy.fcluster(Z, criterion='distance', t = threshold)
        p = m - 1
        rMovies = g == g[p]
        rMovies = np.where(rMovies)[0]
        for possibleMovie in rMovies:
            pM = possibleMovie + 1
            if pM not in favMovies and pM not in alreadyRecommended:
                print('\t\t', pM, moviesT[possibleMovie])
                #print(threshold, pM, moviesT[possibleMovie])
                alreadyRecommended.append(pM)
                foundMovie = True
                break
        if foundMovie: break
        threshold += 0.1

Your favorite movies are:
	Toy Story (1995)
	Batman Forever (1995)
	Free Willy 2: The Adventure Home (1995)
	Star Wars (1977)
	Professional, The (1994)
	Lion King, The (1994)
	Jurassic Park (1993)
	Nightmare Before Christmas, The (1993)
	Aladdin (1992)
	Scream (1996)
	Mis??ables, Les (1995)
	Hercules (1997)

We think you might like:
		 422 Aladdin and the King of Thieves (1996)
		 231 Batman Returns (1992)
		 78 Free Willy (1993)
		 181 Return of the Jedi (1983)
		 129 Bound (1996)
		 99 Snow White and the Seven Dwarfs (1937)
		 222 Star Trek: First Contact (1996)
		 419 Mary Poppins (1964)
		 1219 Goofy Movie, A (1995)
		 895 Scream 2 (1997)
		 1121 Umbrellas of Cherbourg, The (Parapluies de Cherbourg, Les) (1964)
		 63 Santa Clause, The (1994)


# Interesting


- Code that gives you extra information.
- *Not used in the recommendation system.*

In [ ]:
#Gives you the movies that share genre with your fav ones.
for m in favMovies:
    print(moviesT[m - 1])
    r = dfmovies.loc[dfmovies['id'] == m]
    for g in genres:
        print('\t', g, end = ' ')
        if r[g].all(): 
            similarMovies = dfmovies.loc[dfmovies[g] == True]
            print()
            [print('\t\t', i) for i in similarMovies['title']]
        else: print(0)
    print()

In [ ]:
#How many movies we have by genre
for g in genres:
    similarMovies = dfmovies.loc[dfmovies[g] == True]
    print(g, len(similarMovies))

In [ ]:
#Movies that belong to a specific genre
similarMovies = dfmovies.loc[dfmovies['Unknown'] == True]
print(similarMovies['title'])